## Writing Articles with Agentic Workflows
<link rel="stylesheet"
href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css"
integrity="sha384-JcKb8q3iqJ61gNV9KGb8thSsNjpSL0n8PARn9HuZOnIxN0hoP+VmmDGMN5t9UJ0Z"
crossorigin="anonymous">
<div class="bg-info">
<h3>What are agentic workflows?</h3>
</div>

An <b>agentic workflow</b> is a multi-step sequence of tasks or decisions whose orchestration is handled by AI agents. In Generative AI, AI agents are Large Language Models (or "Large Action Models" in this context) that can perform goal-oriented tasks with minimal human intervention. Such tasks may include everything from simple reasoning tasks to complex decision making, and often involve interaction with external resources or systems (called "tools").

The following will build an agentic workflow for writing articles. The initial version of the workflow will consist of the following steps:


1.   A **searcher** agent (LLM + search tool) which will search the web for relevant links based on user input.
2.   An **outliner** node (LLM) which will generate a suitable outline for the article.
3.   A **writer** node (LLM) which will generate the final article.
4.   A **critic** node (LLM) which will critique the writer's output and provide feedback to the **writer** for improvements.








<div class="bg-info">
<h3>What is the difference between an agentic framework and an agentic orchestration framework?</h3>
</div>

There are different schools of thought about agentic orchestration frameworks.

Agentic orchestration can refer to <b>agent-of-agent</b> systems, which are defined based on the number of agents. In this case, they are simply agentic systems that consist of more than a single agent. Hence, it can refer to agentic systems with multiple agents; it can also refer to agentic systems that invoke other agentic systems.

Sometimes, agentic orchestration can also be used to distinguish complex multi-agent systems from simpler agentic systems. In this case, agentic orchestration is only necessary for systems with high levels of complexity. This is based on not just the number of agents, but other factors such as the types of flows (directed versus cyclic).

There are many frameworks that can be used for building agentic workflows. Due to their reasoning and decision-making abilities, LLMs are a natural fit for driving autonomous workflows. However, users often want the ability to extend, constrain or even override aspects of the flow. For example, they may need a way to dynamically limit cycles, manage state across disparate tools, or integrate human-in-the-loop fedback. A popular approach is to use <b>LLM orchestration</b> frameworks. These are frameworks that combine the flexible and dynamic capabilities of agent-driven workflows with low-level control over essential details of the orchestration. This notebook uses <b>LangChain</b> to build the AI agents, and <b>LangGraph</b> to build the agentic workflow that orchestrates the agents.

<div class="bg-info">
<h3>Workflow Summary</h3>
</div>

Steps of the Article Writer:
- <i>Searcher</i> agent receives the article writing request from the user and executes a web search request. It is a tool-based agent, so it uses a tool for its search (indicated by tools in the diagram. The tool it uses is called Tavily - Tavily is a search engine agentic tool.)
  - If it receives a tool invocation request, the workflow sends the request to the Tool Node (Tavily)
  - Else, the workflow sends the search results to the Outliner agent.
- <i>Outliner</i> agent receives the web search results from the Searcher agent and generates the article outline.
  - The workflow sends the outline to the Writer agent.
- <i>Writer</i> agent receives the outline from the Outliner agent (or the Critic agent - see below) and generates an article draft.
  - The workflow sends the draft to the Critic agent.
- <i>Critic</i> agent receives the article draft from the Writer agent and generates feedback about the article for the writer for suggested improvements.
  - If the critic has no suggested improvements, the workflow ends.
  - Else, it sends its suggested improvements back to the writer node.

#Setup

In [3]:
# Install dependencies
!apt install -qq libgraphviz-dev;
!pip install -qU langgraph langgraph langchain_openai langchain_community python-dotenv langchain_mistralai llamaapi langchain-experimental langgraph-checkpoint langgraph-checkpoint-sqlite pygraphviz;

/usr/bin/sh: line 1: apt: command not found
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [98 lines of output]
      /tmp/pip-build-env-9mo5yw7d/overlay/lib/python3.11/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************

In [4]:
# Import libraries
from typing import TypedDict, Literal
import json
import random
from langgraph.graph import END, StateGraph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod
from langchain_core.messages import HumanMessage
import os
from typing import Annotated, Literal, TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
import functools
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time

## Defining Templates
Each agent has its own specific **template**. The templates are defined here.

In [5]:
searcher_template = """
Your job is to search the web for news that would be relevant for generating the article described by the user.

NOTE: Do not write the article. Just search the web for related news if needed and then forward it to the outliner node.
"""

outliner_template = """
Your job is to take as input a list of articles from the web along with instructions from the user on what article they want to write and use that to
generate an outline for the article.
"""

writer_template = """Your job is to write an article using this format:

    TITLE: <title>
    BODY: <body>

NOTE: Do not copy the outline. Just write the article but abide by the outline.
```
"""

critic_template = """Your job is to critique an article written by a writer. Please provide constructive critiques so the writer can improve it.

```GUIDELINES:```

  - Your feedback should be in bullet point format only.
  - The critiques should only focus on are the use of keywords, the title of the article, and the title of the headers, also make sure they include references.
  - NOTE: Do not write the article. Just provide feedback in bullet point format.
  - NOTE: Do not include positive feedback.
  - Never accept the first draft of the article.
  - If you think the article looks good enough, say DONE.
"""

## Defining State
Here, we will define our **GraphState**, as well as the **nodes** and **edges** that our graph is comprised of. This will encapsulate **state** in our agentic workflows.

In [6]:
#####################################
## STATE ##
#####################################
class NMAgentState(TypedDict):
  """
  Encapsulates state in our agentic workflow
  """
  messages: Annotated[list, add_messages]

#####################################
## TOOLS ##
#####################################
google_search_tool = TavilySearchResults(max_results=5, include_answer=True, include_raw_content=True, include_images=True,)

#####################################
## AGENTS ##
#####################################
"""
The LLMs used by the agents
"""
granite_llm = ChatOpenAI(temperature=0,
                         model="granite-3.2-8b-instruct",
                         request_timeout=240)

def create_agent(llm, tools, system_message: str):
    """
    Creates an agent with the given LLM, tools, and system message
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    if tools:
      return prompt | llm.bind_tools(tools)
    else:
      return prompt | llm

searcher_agent = create_agent(granite_llm, [google_search_tool], searcher_template)
outliner_agent = create_agent(granite_llm, [], outliner_template)
writer_agent = create_agent(granite_llm, [], writer_template)
critic_agent = create_agent(granite_llm, [], critic_template)

#####################################
## NODES ##
#####################################
def agent_node(state, agent, name):
  result = agent.invoke(state)
  return { "messages": [result] }

searcher_node = functools.partial(agent_node, agent=searcher_agent, name="Search Agent")
outliner_node = functools.partial(agent_node, agent=outliner_agent, name="Outliner Agent")
writer_node = functools.partial(agent_node, agent=writer_agent, name="Writer Agent")
tool_node = ToolNode([google_search_tool])
critic_node = functools.partial(agent_node, agent=critic_agent, name="Critic Agent")

#####################################
## EDGES ##
#####################################
def should_search(state) -> Literal['tools', 'outliner']:
  if len(state['messages']) and state['messages'][-1].tool_calls:
    return "tools"
  else:
    return "outliner"

def should_edit(state) -> Literal['writer', END]:
  if len(state['messages']) and 'DONE' in state['messages'][-1].content:
    return END
  else:
    return "writer"

/tmp/ipykernel_14258/1807005728.py:13: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  google_search_tool = TavilySearchResults(max_results=5, include_answer=True, include_raw_content=True, include_images=True,)


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Defining the Workflow Graph
Here, we will define the workflow, which will encapsulate the state, nodes and edges defined above.

In [ ]:
workflow = StateGraph(NMAgentState)

# nodes
workflow.add_node("searcher", searcher_node)
workflow.add_node("outliner", outliner_node)
workflow.add_node("writer", writer_node)
workflow.add_node("tools", tool_node)
workflow.add_node("critic", critic_node)

# entrypoint
workflow.set_entry_point("searcher")

# edges
workflow.add_conditional_edges("searcher", should_search)
workflow.add_edge("tools", "searcher")
workflow.add_edge("outliner", "writer")
workflow.add_edge("writer", 'critic')
workflow.add_conditional_edges("critic", should_edit)

# compile the workflow into a graph
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer, interrupt_before=['critic'])

Visualize the graph:

In [ ]:
display(Image(graph.get_graph().draw_png()))

# Testing the workflow
Now that the workflow has been generated, we can test it out with different prompts.

In [ ]:
# Prompt to test
input = "Generate an article about Cohesity's Global Support and Services."

In [ ]:
config = {"configurable": {"thread_id": 12, "recursion_limit": 10}}
try:
  for event in graph.stream({"messages": [HumanMessage(content=input)]}, config, stream_mode="values"):
      event['messages'][-1].pretty_print()
except Exception as e:
  print(f"\n\nErrors generating response:\n===============\n {str(e)}")